<a href="https://colab.research.google.com/github/Cole-Pd/DeepDive/blob/main/project_6_draft_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 6

## Problem Definition


State the business problem. Translate the business problem into a Data Science problem by stating what kind of problem it is ( supervised vs unsupervised ) and whether it is a classification, regression, or clustering problem.

## Data Collection/Sources


In [1]:
import pickle
import pandas as pd
import tensorflow.keras as keras
import numpy as np
from numpy import asarray
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.utils import plot_model
from sklearn.model_selection import train_test_split
import seaborn as sns
from PIL import Image
import imageio.v2 as img

In [13]:
x_pickle = !curl -O 'https://ddc-datascience.s3.amazonaws.com/Projects/Project.6-Images/Data/X.pickle'

In [14]:
y_pickle = !curl -O 'https://ddc-datascience.s3.amazonaws.com/Projects/Project.6-Images/Data/y.pickle'


* Load pickled data from X.pickle and y.pickle from the AWS S3 bucket.



In [15]:
# Feature set of Dogs and Cats
#x_pickle = '/content/X.pickle'
train_set = pickle.load(open('X.pickle','rb'))

In [16]:
# Target set of Dogs and Cats
y_pickle = '/content/y.pickle'
target_set = pickle.load( open('y.pickle','rb'))

## Data Cleaning



* Scale the values in X so that they fall between 0 and 1 by dividing by 255.



In [17]:
train_set = (train_set)/255

In [18]:
group = []
for i in range(len(train_set)):
  group.append(train_set[i].shape)

In [19]:
pd.Series(group).value_counts()

(100, 100, 1)    24946
dtype: int64

## Exploratory Data Analysis


* Look at the shape of X and y. Ensure that X is 4 dimensional.

In [ ]:
train_set.shape

(24946, 100, 100, 1)

In [ ]:
train_set[0].shape

(100, 100, 1)

In [ ]:
target_set = np.array(target_set)

In [ ]:
type(train_set)

numpy.ndarray

* Plot one of the images in X using plt.imshow().


In [ ]:
# plt.imshow(train_set[38], 'Greys') ;

* Look at the response value in y for that image.

In [ ]:
target_set[341] # values of 1 are cat(s) values of 0 are dog(s).

1

## Processing


* Split X and y into training and testing sets.

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(train_set, target_set, test_size = .2)

*  Build a neural network:

In [ ]:
model = Sequential()

In [ ]:
# Add Convolutional and MaxPooling layers.
model.add(
  Conv2D(
    name = "Layer_1",
    input_shape = xtrain.shape[1:],
    filters = 30,
    kernel_size = (3,3),
    activation = 'relu', #Relu is linear.
  )
)
model.add(
    MaxPooling2D(pool_size=(2,2))
)

model.add(
  Conv2D(
    name = "Layer_2",
    input_shape = xtrain.shape[1:],
    filters = 30,
    kernel_size = (3,3),
    activation = 'relu',
  )
)

model.add(
    MaxPooling2D(pool_size=(2,2))
)

model.add(
  Conv2D(
    name = "Layer_3",
    input_shape = xtrain.shape[1:],
    filters = 30,
    kernel_size = (3,3),
    activation = 'relu',
  )
)

model.add(
    MaxPooling2D(pool_size=(2,2))
)


In [ ]:
# Flatten and add a Dense Layer
model.add(
    Flatten()
)

model.add(
  Dense(
    name = 'binary output',
    units = 1,
    activation = 'sigmoid' #Sigmoid is binary
  )
)

In [ ]:
# Compile model
model.compile(
  optimizer = 'adam',
  loss = 'binary_crossentropy',
  metrics = ['accuracy'],
)

* Fit your model using the training set.

In [ ]:
model.fit(xtrain, ytrain, epochs = 15)

Epoch 1/15
624/624 [==============================] - 6s 9ms/step - loss: 0.6293 - accuracy: 0.6393
Epoch 2/15
624/624 [==============================] - 5s 8ms/step - loss: 0.5258 - accuracy: 0.7377
Epoch 3/15
624/624 [==============================] - 5s 8ms/step - loss: 0.4590 - accuracy: 0.7853
Epoch 4/15
624/624 [==============================] - 5s 9ms/step - loss: 0.4264 - accuracy: 0.8053
Epoch 5/15
624/624 [==============================] - 5s 8ms/step - loss: 0.3988 - accuracy: 0.8191
Epoch 6/15
624/624 [==============================] - 6s 9ms/step - loss: 0.3763 - accuracy: 0.8325
Epoch 7/15
624/624 [==============================] - 5s 8ms/step - loss: 0.3489 - accuracy: 0.8457
Epoch 8/15
624/624 [==============================] - 5s 8ms/step - loss: 0.3311 - accuracy: 0.8552
Epoch 9/15
624/624 [==============================] - 5s 9ms/step - loss: 0.3107 - accuracy: 0.8680
Epoch 10/15
624/624 [==============================] - 6s 10ms/step - loss: 0.2981 - accuracy: 0.873

156/156 [==============================] - 1s 4ms/step - loss: 0.3768 - accuracy: 0.8501
85.01%


In [ ]:
from sklearn.metrics import confusion_matrix

ypredicted = model.predict(xtest) > 0.5
matrix = confusion_matrix(ytest, ypredicted)
labels = ['Dog', 'Cat']

sns.heatmap(matrix, square=True, annot=True, fmt='d', cbar=False, cmap='Blues',
            xticklabels=labels, yticklabels=labels)

plt.xlabel('Predicted label') ;
plt.ylabel('Actual label') ;
plt.title('Confusion') ;

loss, accuracy = model.evaluate(xtest, ytest)
print(f'{round(accuracy*100,2)}%')

## Data Visualization/Communication of Results


In [ ]:
from PIL import Image
from numpy import asarray

In [ ]:
good_cone_boy = img.imread('https://ddc-datascience.s3.amazonaws.com/Projects/Project.6-Images/Data/dog.jpg')
transform_gcb = np.array(Image.fromarray(good_cone_boy).convert('L').resize([100,100]))
transform_gcb = transform_gcb.reshape((1,) + transform_gcb.shape)

In [ ]:
our_prediction = model.predict(transform_gcb)
if int(our_prediction[0][0]) == 0:
  print('We predict the image is a dog :)')
else:
  print('We predict the image is a cat :)')

1/1 [==============================] - 0s 122ms/step
We predict the image is a dog :)


In [ ]:
def cat_dog_predictor(url_to_jpg = (str)):
  ''' Using the predictive model we wrote in this notebook we will
  transform a jpg image to a shape and see make a prediction of if it
  is a dog or cat. Note: It will make the same prediction for jpgs that
  are well not dogs or cats. This model works for individual jpgs of tensor 2.
  '''

  t2_array = img.imread(url_to_jpg)
  gs_resize_array = np.array(Image.fromarray(t2_array).convert('L').resize([100,100]))
  transform_array = gs_resize_array.reshape((1,) + gs_resize_array.shape)
  our_prediction = model.predict(transform_array) ;

  if int(our_prediction[0][0]) == 0:
    return 'We predict the image is a dog :)'
  else:
    return 'We predict the image is a cat :)'


In [ ]:
is_cat = 'https://i.pinimg.com/originals/81/6d/a5/816da533638aee63cfbd315ea24cccbd.jpg'
cat_dog_predictor(is_cat)

1/1 [==============================] - 0s 19ms/step


'We predict the image is a cat :)'

In [ ]:
# def cats_and_dogs_predictor(list_of_urls = (list)):
#   ''' This method takes ursl
#   Using the predictive model we wrote in this notebook we will
#   transform a jpg image to a shape and see make a prediction of if it
#   is a dog or cat. Note: It will make the same prediction for jpgs that
#   are well not dogs or cats. This model works for individual jpgs of tensor 2.
#   '''

#   nuggets = {}
#   try:

#     for nugget in list_of_urls:
#       t2_array = img.imread(nugget)
#       gs_resize_array = np.array(Image.fromarray(t2_array).convert('L').resize([100,100]))
#       transform_array = gs_resize_array.reshape((1,) + gs_resize_array.shape)
#       our_prediction = model.predict(transform_array) ;

#     if int(our_prediction[0][0]) == 1:
#       nuggets[nugget] = 'Probably a Dog :)'
#     else:
#       nuggets[nugget] = 'Probably a Cat :)'

#   except:
#     nuggets[nugget] = "This one's unpredictable :)"

#   return pd.DataFrame(nuggets)

In [ ]:
cats_list = [
    "https://www.bluecross.org.uk/sites/default/files/assets/images/124044lpr.jpg",
    "https://static.boredpanda.com/blog/wp-content/uploads/2018/04/5acb63d83493f__700-png.jpg",
    "https://www.argospetinsurance.co.uk/assets/uploads/2017/12/cat-pet-animal-domestic-104827.jpeg",
    "https://www.pets4homes.co.uk/images/articles/771/large/cat-lifespan-the-life-expectancy-of-cats-568e40723c336.jpg",
    "http://www.catster.com/wp-content/uploads/2017/08/A-fluffy-cat-looking-funny-surprised-or-concerned.jpg",
    "https://www.cats.org.uk/uploads/images/featurebox_sidebar_kids/grief-and-loss.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Cat03.jpg/1200px-Cat03.jpg",
    "https://www.petmd.com/sites/default/files/petmd-cat-happy-13.jpg",
    "https://r.hswstatic.com/w_907/gif/tesla-cat.jpg",
    "https://images.pexels.com/photos/617278/pexels-photo-617278.jpeg?auto=compress&cs=tinysrgb&h=350",
    "http://www.catster.com/wp-content/uploads/2017/08/Pixiebob-cat.jpg",
    "http://www.petmd.com/sites/default/files/what-does-it-mean-when-cat-wags-tail.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/7/74/Cat-eating-prey.jpg/220px-Cat-eating-prey.jpg",
    "https://i.ytimg.com/vi/YCaGYUIfdy4/maxresdefault.jpg",
    "https://www.prestigeanimalhospital.com/sites/default/files/08-cat-cancer-4.jpeg",
    "https://metrouk2.files.wordpress.com/2017/11/capture16.png?w=748&h=706&crop=1",
    "https://i.pinimg.com/originals/81/6d/a5/816da533638aee63cfbd315ea24cccbd.jpg",
    "https://img.purch.com/w/660/aHR0cDovL3d3dy5saXZlc2NpZW5jZS5jb20vaW1hZ2VzL2kvMDAwLzA5Ny85NTkvb3JpZ2luYWwvc2h1dHRlcnN0b2NrXzYzOTcxNjY1LmpwZw==",
    "https://www.telegraph.co.uk/content/dam/pets/2017/01/06/1-JS117202740-yana-two-face-cat-news_trans_NvBQzQNjv4BqJNqHJA5DVIMqgv_1zKR2kxRY9bnFVTp4QZlQjJfe6H0.jpg?imwidth=450",
    "https://img-aws.ehowcdn.com/600x600p/photos.demandstudios.com/getty/article/165/76/87490163.jpg",
    "http://catsatthestudios.com/wp-content/uploads/2017/12/12920541_1345368955489850_5587934409579916708_n-2-960x410.jpg",
    "http://www.petmania.ie/images/default-source/cat/petmania-kitten-1.jpg?sfvrsn=4",
    "https://imagesvc.timeincapp.com/v3/mm/image?url=https%3A%2F%2Fpeopledotcom.files.wordpress.com%2F2018%2F04%2Floki_the_sphynx-01_1.jpg%3Fw%3D1800&w=700&q=85",
    "https://pbs.twimg.com/profile_images/378800000532546226/dbe5f0727b69487016ffd67a6689e75a_400x400.jpeg",
    "http://theorphanpet.com/wp-content/uploads/2016/06/13233136_788779574558512_3397930498291044113_n.jpg",
    "http://r.ddmcdn.com/s_f/o_1/cx_462/cy_245/cw_1349/ch_1349/w_720/APL/uploads/2015/06/caturday-shutterstock_149320799.jpg",
    "https://ichef.bbci.co.uk/images/ic/720x405/p0517py6.jpg",
    "https://peopledotcom.files.wordpress.com/2017/12/smush-the-cat-1.jpg",
    "http://i0.kym-cdn.com/entries/icons/original/000/002/232/bullet_cat.jpg",
    "http://r.ddmcdn.com/s_f/o_1/cx_0/cy_157/cw_1327/ch_1327/w_720/APL/uploads/2013/01/smart-cat-article.jpg",
    "https://images.pexels.com/photos/326875/pexels-photo-326875.jpeg?cs=srgb&dl=adorable-animal-blur-326875.jpg&fm=jpg",
    "http://www.catster.com/wp-content/uploads/2017/08/Hairless-cat.jpg",
    "https://www.vetbabble.com/wp-content/uploads/2016/11/hiding-cat.jpg",
    "http://www.catbreedslist.com/cat-wallpapers/Persian-kitten-grass-white-2560x1600.jpg",
    "https://bayonline.co.nz/wp-content/uploads/2016/10/4-ways-cheer-up-depressed-cat.jpg",
    "https://i.ytimg.com/vi/OqQPv78AMw0/maxresdefault.jpg",
    "https://lookaside.fbsbx.com/lookaside/crawler/media/?media_id=1915279545376970",
    "https://localtvkstu.files.wordpress.com/2018/02/turtle-the-cat-1.jpg?quality=85&strip=all&w=2000",
    "https://static.designboom.com/wp-content/uploads/2018/04/cat-VR-isobar-designboom-600.jpg",
    "https://i.kinja-img.com/gawker-media/image/upload/s--kHrQ8nr7--/c_scale,f_auto,fl_progressive,q_80,w_800/18huxz4bvnfjbjpg.jpg",
    "https://lookaside.fbsbx.com/lookaside/crawler/media/?media_id=1394987587246423",
    "https://cbsnews1.cbsistatic.com/hub/i/r/2011/09/30/91f6cb1f-a643-11e2-a3f0-029118418759/resize/620x465/30aeb6468b2b25d07324e6d4ef3c9b47/twofacedcat_AP110928014"]

In [ ]:
def cats_and_dogs_predictor(list_of_urls = (list)):
  ''' This method takes ursl
  Using the predictive model we wrote in this notebook we will
  transform a jpg image to a shape and see make a prediction of if it
  is a dog or cat. Note: It will make the same prediction for jpgs that
  are well not dogs or cats. This model works for individual jpgs of tensor 2.
  '''

  predictionary = {}

  cat_nuggets = []
  dog_nuggets = []
  unpredictable_nuggets = []

  for nugget in list_of_urls:
    try:
      t2_array = img.imread(nugget)
      gs_resize_array = np.array(Image.fromarray(t2_array).convert('L').resize([100,100]))
      transform_array = gs_resize_array.reshape((1,) + gs_resize_array.shape)
      our_prediction = model.predict(transform_array) ;

      if int(our_prediction[0][0]) == 0:
        dog_nuggets.append(nugget)
      else:
        cat_nuggets.append(nugget)

    except:
      unpredictable_nuggets.append(nugget)

  cat_nuggets = pd.Series(cat_nuggets)
  dog_nuggets = pd.Series(dog_nuggets),
  unpredictable_nuggets = pd.Series(unpredictable_nuggets)

  predictionary['Cats'] = cat_nuggets
  predictionary['Dogs'] = dog_nuggets
  predictionary['Unpredictable'] = unpredictable_nuggets

  return predictionary, 100 * (
     len(cat_nuggets) /
   (len(cat_nuggets) + len(dog_nuggets))  )



In [ ]:
cats_and_dogs_predictor(cats_list)